# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
import time, gc

In [6]:
from itertools import product


In [7]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [8]:
from util import util_search

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


In [9]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [10]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [11]:
PATH_SEARCH_EXPERIMENT =  '../data/search/juris_tcu_index/search_experiment_juris_tcu_index.csv'
PATH_SEARCH_RESULT =  '../data/search/juris_tcu_index/search_experiment_result_juris_tcu_index.csv'

# Data load

## Query data load

In [12]:
df_query = pd.read_csv(PATH_QUERY)

In [13]:
df_query.shape

(16022, 11)

In [14]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,34899,"A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perd...","Lei Ordinária 8.443/1992, art. 58, inciso II",NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 2669/2012 - Plenário,2012-10-03,RELATOR,JOSÉ JORGE
1,30271,"A contratação de médicos e profissionais da área de saúde, como colaboradores eventuais, com pag...",NaN,NaN,Pessoal,1131,REPRESENTAÇÃO,Acórdão 2669/2012 - Plenário,2012-10-03,RELATOR,JOSÉ JORGE
2,26574,"Para que seja conhecido o recurso de revisão, não basta apenas que se apresente documento ainda ...",NaN,NaN,Direito processual,5288,TOMADA DE CONTAS,Acórdão 514/2013 - Plenário,2013-03-13,RELATOR,ANA ARRAES
3,17902,"A contratação de serviços comuns de engenharia que possam ser objetivamente definidos em edital,...",NaN,NaN,Licitação,932,RELATÓRIO DE LEVANTAMENTO,Acórdão 3144/2012 - Plenário,2012-11-21,RELATOR,ANA ARRAES
4,26089,"A Fundação Banco do Brasil, por receber recursos da União, deve observar, quando do repasse de r...",NaN,NaN,Competência do TCU,5095,TOMADA DE CONTAS ESPECIAL,Acórdão 2071/2013 - Plenário,2013-08-07,RELATOR,JOSÉ JORGE


## Qrel data load

In [15]:
df_qrel = pd.read_csv(PATH_QREL)

In [16]:
df_qrel.shape

(94653, 3)

In [17]:
df_qrel.head()

,ID_QUERY,ID_DOCTO,TYPE
0,158,15147,INDEXACAO_EXTRA
1,37,15147,INDEXACAO_EXTRA
2,178,15147,INDEXACAO_EXTRA
3,14564,15147,INDEXACAO_EXTRA
4,9219,15147,INDEXACAO_EXTRA


In [18]:
df_qrel.shape[0]

94653

In [19]:
df_search_data = df_query.merge(df_qrel, how='left', left_on='ID', right_on='ID_QUERY').drop('ID_QUERY', axis=1)

In [20]:
df_search_data.shape[0]


94653

In [21]:
# Agrupar os dados pelo ID e criar a coluna 'RELEVANCE_LIST' com a lista de tuplas
# df_new = df_search_data.groupby('ID').apply(lambda x: list(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_LIST')
# df_new['RELEVANCE_LIST'] = df_new['RELEVANCE_LIST'].apply(lambda x: sorted(x, key=lambda tup: ('AREA', 'TEMA', 'SUBTEMA', 'INDEXACAO_EXTRA').index(tup[1])))
df_new = df_search_data.groupby('ID').apply(lambda x: dict(zip(x['ID_DOCTO'], x['TYPE']))).reset_index(name='RELEVANCE_DICT')


In [22]:
df_new.head()

,ID,RELEVANCE_DICT
0,5,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,6,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,7,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,8,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,9,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [23]:
# Unir o resultado com as demais colunas únicas
df_new = pd.merge(df_new, df_search_data.drop_duplicates('ID'), on='ID', how='left')

In [24]:

# Selecionar as colunas desejadas
df_search_data = df_new[['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE', 'NORMATIVE_AUTHOR_NAME', 'RELEVANCE_DICT']]


In [25]:
df_search_data.shape

(16022, 12)

In [26]:
df_search_data = df_search_data.reset_index()

In [27]:
df_search_data.head()

,index,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,RELEVANCE_DICT
0,0,5,"SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial co...","Lei Ordinária 3738/1960 || Lei Ordinária 1711/1952, art. 184",SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{1298: 'INDEXACAO_EXTRA', 15340: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106..."
1,1,6,"SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fund...",Lei Ordinária 3738/1960,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{207: 'INDEXACAO_EXTRA', 15961: 'INDEXACAO_EXTRA', 1131: 'AREA', 5106: 'SUBTEMA', 1113: 'TEMA'}"
2,2,7,SÚMULA TCU 3: O arquivamento é a solução indicada para as hipóteses em que as contas de responsá...,NaN,SUMULA,Direito processual,5288,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{108: 'INDEXACAO_EXTRA', 1526: 'TEMA', 1727: 'SUBTEMA', 223: 'INDEXACAO_EXTRA', 650: 'INDEXACAO_..."
3,3,8,"SÚMULA TCU 4: A reclassificação de cargos não aproveita ao servidor aposentado, a menos que lei ...",NaN,SUMULA,Pessoal,1131,NaN,Ata 88/1973 - Plenário,1973-12-04,RELATOR,OCTÁVIO GALLOTTI,"{887: 'SUBTEMA', 92: 'TEMA', 1131: 'AREA', 1100820: 'INDEXACAO_EXTRA'}"
4,4,9,"SÚMULA TCU 5 (REVOGADA): As sociedades de economia mista, salvo disposição expressa em lei, não ...",NaN,SUMULA,Competência do TCU,5095,NaN,AC 2082/2007-PL,2007-10-03,RELATOR,UBIRATAN AGUIAR,"{1215: 'INDEXACAO_EXTRA', 14715: 'SUBTEMA', 46: 'TEMA', 1480: 'INDEXACAO_EXTRA', 5095: 'AREA'}"


In [28]:
del df_new

# Create reference to index (elastic search)

Documentation https://docs.haystack.deepset.ai/docs
https://haystack.deepset.ai/tutorials/09_dpr_training 


In [29]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'XqjmOmuaQxqmmxmdE65Q2Q', 'pri': '1', 'rep': '1', 'docs.count': '13252', 'docs.deleted': '13252', 'store.size': '289.2mb', 'pri.store.size': '289.2mb'}



In [30]:
index_dict

{'indir_juris_tcu_index': {'health': 'yellow',
  'status': 'open',
  'index': 'indir_juris_tcu_index',
  'uuid': 'XqjmOmuaQxqmmxmdE65Q2Q',
  'pri': '1',
  'rep': '1',
  'docs.count': '13252',
  'docs.deleted': '13252',
  'store.size': '289.2mb',
  'pri.store.size': '289.2mb'}}

In [31]:
INDEX_NAME = 'indir_juris_tcu_index'

In [32]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13252

Qtd de embeddings 13252

Documento.id=1: <Document: id=1, content='O termo é "Abandono de cargo".
Abandono de cargo tem definição: "Configura abandono de cargo a ausên...'>


# Pipelines creation

In [33]:
pipes = []

In [34]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [35]:
raise Exception ('Stop execution - create only desired pipelines in code below ')

Exception: Stop execution - create only desired pipelines in code below 

## First stage = BM25

### pipe_bm25_ranker_monot5_3b

In [36]:
nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")
nome_caminho_modelo_monot5_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_monot5_3b), f"Path para {nome_caminho_modelo_monot5_3b} não existe!"

In [37]:
pipe_bm25_ranker_monot5_3b = util_search.return_pipeline_bm25_reranker(index, 'MONOT5', nome_caminho_modelo_monot5_3b, parm_limit_query_size=350)

In [38]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_bm25_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'bm25',  # or 'sts'
              'RETRIEVER_MODEL_NAME': '', # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [39]:
doctos_retornados_ranker = pipe_bm25_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1104189, content='O termo é "Processo administrativo de responsabilização". Processo administrativo de responsabilizaç...'>

Último (10):
<Document: id=1102005, content='O termo é "Plano de contratações".
Plano de contratações tem definição: "É o documento no qual a org...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Processo administrativo de responsabilização', -1.0603076219558716]
1 : ['Competência do TCU', -1.4214444160461426]
2 : ['Evidência documental', -1.9571717977523804]
3 : ['Ofício de requisição', -2.266594648361206]
4 : ['Julgamento de contas', -2.5948143005371094]
5 : ['Nexo de causalidade', -3.0983963012695312]
6 : ['Teletrabalho'

### pipe_bm25_ranker_minilm

In [ ]:
nome_modelo_ranking_minilm = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_minilm = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_minilm
assert os.path.exists(nome_caminho_modelo_minilm), f"Path para {nome_caminho_modelo_minilm} não existe!"

In [ ]:
pipe_bm25_ranker_minilm = util_search.return_pipeline_bm25_reranker(index, 'MINILM', nome_caminho_modelo_minilm, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker_minilm',
              'PIPE_OBJECT': pipe_bm25_ranker_minilm,
              'RETRIEVER_TYPE': 'bm25',  # or 'sts'
              'RETRIEVER_MODEL_NAME': '', # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_ranking_minilm})

In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker_minilm.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU". Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (10):
<Document: id=4495, content='O termo é "Nexo de causalidade".
Nexo de causalidade tem definição: "Elemento que evidencia se a con...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', 0.0014646538766101003]
1 : ['Processo administrativo de responsabilização', 0.0004114470211789012]
2 : ['Evidência documental', 0.00033812460605986416]
3 : ['Plano de contratações', 0.0002457168884575367]
4 : ['Julgamento de contas', 0.00022130725847091526]
5 : ['Teletrabalho', 0.00016929447883740067]
6 : ['Ofício de

## First stage = Sentence Similarity

### pipe_sts_ranker_minilm

In [ ]:
nome_modelo_embedding_model_sts = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo_sts = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts
assert os.path.exists(nome_caminho_modelo_sts), f"Path para {nome_caminho_modelo_sts} não existe!"


In [ ]:
pipe_sts_ranker_minilm = util_search.return_pipeline_sts_reranker(index, 'MINILM', parm_path_model_ranker=nome_caminho_modelo_minilm, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_sts_ranker_minilm',
              'PIPE_OBJECT': pipe_sts_ranker_minilm,
              'RETRIEVER_TYPE': 'sts',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_ranking_minilm})

In [ ]:
doctos_retornados_ranker = pipe_sts_ranker_minilm.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1110487, content='O termo é "Vazamento de dados".
Vazamento de dados tem definição: "Transmissão não-autorizada de dad...'>

Último (10):
<Document: id=15939, content='O termo é "Erro de procedimento".
Erro de procedimento tem definição: "É um vício de forma, extrínse...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Vazamento de dados', 0.0024509401991963387]
1 : ['Termo de sigilo', 0.0007997120846994221]
2 : ['Trancamento das contas', 0.0002845456183422357]
3 : ['Risco de controle', 0.00022979704954195768]
4 : ['Revisão de ofício', 0.00013715452223550528]
5 : ['Responsabilidade perante o controle externo', 0.00013104191748425364]
6 : ['Anulação

### pipe_sts_ranker_monot5_3b

In [ ]:
pipe_sts_ranker_monot5_3b = util_search.return_pipeline_sts_reranker(index, 'MONOT5', parm_path_model_ranker=nome_caminho_modelo_monot5_3b, parm_path_model_sts=nome_caminho_modelo_sts, parm_limit_query_size=350)

In [ ]:
pipes.append({'PIPE_NAME': 'pipe_sts_ranker_monot5_3b',
              'PIPE_OBJECT': pipe_sts_ranker_monot5_3b,
              'RETRIEVER_TYPE': 'sts',  # in ['sts', 'bm25']
              'RETRIEVER_MODEL_NAME': nome_modelo_embedding_model_sts, # or nome_modelo_embedding_model_sts_rufimelo,
              'RANKER_MODEL_NAME': nome_modelo_monot5_3b})

In [ ]:
doctos_retornados_ranker = pipe_sts_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=2663, content='O termo é "Trancamento das contas".
Trancamento das contas tem definição: "O Tribunal ordenará o tra...'>

Último (10):
<Document: id=83, content='O termo é "Anulação".
Anulação tem definição: "É o ato ou a decisão, de caráter judicial ou administ...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Trancamento das contas', -6.416285991668701]
1 : ['Afastamento de responsável', -6.9923858642578125]
2 : ['Termo de sigilo', -8.37442398071289]
3 : ['Responsabilidade perante o controle externo', -8.556228637695312]
4 : ['Revisão de ofício', -9.356241226196289]
5 : ['Vazamento de dados', -9.744372367858887]
6 : ['Ato vinculado', -9.8531026

# Experiments

Rodar monoT5_3b: bm25

In [40]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [82]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [300, 200, 100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [83]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [84]:
print(len(list_experiment))
gc.collect()

6


103

In [85]:
list_experiment


[{'CRITERIA': 'total',
  'TOPK_RETRIEVER': 300,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 {'CRITERIA': 'total',
  'TOPK_RETRIEVER': 200,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 {'CRITERIA': 'total',
  'TOPK_RETRIEVER': 100,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 

In [87]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [88]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, 
                                    parm_path_experiment= PATH_SEARCH_EXPERIMENT,
                                    parm_path_experiment_result= PATH_SEARCH_RESULT)



===0/6===>  2023-Jun-11 19:44:36 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 11%|█         | 11/100 [03:44<29:46, 20.08s/it]

Rodar minilm: bm25 e sts

In [ ]:
pipes

[{'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
  'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
  'RETRIEVER_TYPE': 'bm25',
  'RETRIEVER_MODEL_NAME': '',
  'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}]

In [ ]:
grid_experiment = {'CRITERIA' : ['total', 'total_gte_5'],
             'TOPK_RETRIEVER' : [300, 200, 100],
             'TOPK_RANKER' : [100],
             'PIPE': pipes,
             'DONE': [False]
}

In [ ]:
list_experiment = [dict(zip(grid_experiment.keys(), values)) for values in product(*grid_experiment.values())]


In [ ]:
print(len(list_experiment))
gc.collect()

6


103

In [ ]:
list_experiment


[{'CRITERIA': 'total',
  'TOPK_RETRIEVER': 300,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 {'CRITERIA': 'total',
  'TOPK_RETRIEVER': 200,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 {'CRITERIA': 'total',
  'TOPK_RETRIEVER': 100,
  'TOPK_RANKER': 100,
  'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b',
   'PIPE_OBJECT': <haystack.pipelines.base.Pipeline at 0x7fd807582b50>,
   'RETRIEVER_TYPE': 'bm25',
   'RETRIEVER_MODEL_NAME': '',
   'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'},
  'DONE': False},
 

In [ ]:
import importlib
importlib.reload(util_search)

dict_idcg_relevance_fixed {1: 1.0, 2: 1.6309297535714575, 3: 2.1309297535714578, 4: 2.5616063116448506, 5: 2.9484591188793923, 6: 3.3046663059874146, 7: 3.637999639320748, 8: 3.953464516106477, 9: 4.254494511770458, 10: 4.543559338088346, 11: 4.8225022837394755, 12: 5.092740438166795, 13: 5.355389973203989, 14: 5.611347998013804}


<module 'util.util_search' from '/home/borela/fontes/ind-ir/code/util/util_search.py'>

In [ ]:
%%time
# colocar pequeno só para testes, depois voltar para 9999999 (número maior do que o número de termos a pesquisar)
limit_query = 100 # 9999999
for cnt, experiment in enumerate(list_experiment):
    if not experiment['DONE']:
        list_result_experiment = [] # por experiment
        # no caso de normas segecex, bastaria 224 para qtd5 e 891 para qtd1
        print(f"==={cnt}/{len(list_experiment)}===>  {time.strftime('%Y-%b-%d %H:%M:%S')} experiment: {experiment}")
        result_run = util_search.experiment_run(parm_df=df_search_data, 
                                                parm_experiment=experiment,
                                                parm_ndcg_position=12,
                                                parm_limit_query=limit_query,
                                                parm_print=True)
        list_result_experiment.append(result_run)
        # print(f"qtd_encontrado sim:{result_run['qtd_encontrado']}, não:{result_run['qtd_nao_encontrado']} ({result_run['percent_nao_encontrado']}%)")
        experiment['DONE'] = True
        util_search.add_experiment_result(parm_list_result=list_result_experiment, 
                                    parm_path_experiment= PATH_SEARCH_EXPERIMENT,
                                    parm_path_experiment_result= PATH_SEARCH_RESULT)



===0/6===>  2023-Jun-11 19:44:36 experiment: {'CRITERIA': 'total', 'TOPK_RETRIEVER': 300, 'TOPK_RANKER': 100, 'PIPE': {'PIPE_NAME': 'pipe_bm25_ranker_monot5_3b', 'PIPE_OBJECT': <haystack.pipelines.base.Pipeline object at 0x7fd807582b50>, 'RETRIEVER_TYPE': 'bm25', 'RETRIEVER_MODEL_NAME': '', 'RANKER_MODEL_NAME': 'unicamp-dl/mt5-3B-mmarco-en-pt'}, 'DONE': False}


 11%|█         | 11/100 [03:44<29:46, 20.08s/it]

Rodar monoT5_3b: sts